In [172]:
import pandas as pd
key_words = ['INDI', 'NAME', 'SEX', 'BIRT', 'DEAT', 'FAMC', 'FAMS', 'FAM', 'MARR', 'HUSB', 'WIFE', 'CHIL', 'DIV', 'DATE', 'HEAD', 'TRLR', 'NOTE']

In [173]:
file = open('example.ged', 'r')

In [174]:
# print the unique identifiers and names of each of the individuals 
# in order by their unique identifiers. 
individuals = pd.DataFrame(columns=['ID', 'NAME', 'SEX', 'BIRT', 'DEAT', 'FAMC', 'FAMS'])
families = pd.DataFrame(columns=['ID', 'MARR', 'DIV', 'HUSB', 'WIFE', 'CHIL'])

curr_id = ''
isIndividual = False
isBirth = True

for line in file:
    line_words = line.split()
    if not line_words:
        continue
    # get the first element since at is the tag of line
    level = line_words[0].strip()
    try:
        if line_words[2].strip() == "INDI" or line_words[2].strip() == "FAM":
            if line_words[2] == "INDI":
                isIndividual = True
            else:
                isIndividual = False
            line_tag = line_words[2].strip()
            arguments = line_words[1].strip()
      
        else:
            line_tag = line_words[1].strip()
            arguments = line.split(line_tag)[-1].strip() 
    except:
        line_tag = line_words[1].strip()
        arguments = line.split(line_tag)[-1].strip()
    
    # check if that is present in the keywords
    if line_tag in key_words:
        if (isIndividual):
            if level == "0": # new block, individual or fam
                curr_id = arguments
                new_row = pd.DataFrame([{'ID': curr_id, 'NAME':pd.NA, 'SEX':pd.NA, 'BIRT':pd.NA, 'DEAT':pd.NA, 'FAMC':[], 'FAMS':[]}])
                individuals = pd.concat([individuals, new_row], ignore_index=True)
            if level == "1" and (line_tag != 'BIRT' or line_tag!= 'DEAT' or line_tag!="INDI"):
                index_to_modify = individuals[individuals['ID'] == curr_id].index[0]
                if line_tag == "FAMC" or line_tag == "FAMS":
                    individuals.loc[index_to_modify, line_tag].append(arguments)
                else:
                    individuals.loc[index_to_modify, line_tag] = arguments
            if level == "2":
                index_to_modify = individuals[individuals['ID'] == curr_id].index[0]
                if isBirth:
                    individuals.loc[index_to_modify, 'BIRT'] = arguments
                else:
                    individuals.loc[index_to_modify, 'DEAT'] = arguments
            else:
                if line_tag == "BIRT":
                    isBirth = True
                elif line_tag == "DEAT":
                    isBirth = False
    else:
        valid = 'N'
print(individuals)

     ID  NAME   SEX         BIRT        DEAT    FAMC          FAMS
0  @I1@  <NA>  <NA>   9 SEP 1999        <NA>  [@F3@]  [@F1@, @F2@]
1  @I2@  <NA>  <NA>  12 MAR 1973        <NA>      []        [@F3@]
2  @I3@  <NA>  <NA>   4 MAY 1975        <NA>      []        [@F3@]
3  @I4@  <NA>  <NA>   4 MAR 1999  8 MAY 2020      []        [@F2@]
4  @I5@  <NA>  <NA>   4 FEB 1999        <NA>      []        [@F1@]
5  @I6@  <NA>  <NA>   1 JAN 2023        <NA>  [@F1@]            []
6  @I7@  <NA>  <NA>   9 FEB 2019        <NA>  [@F2@]            []
